In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp  ./drive/MyDrive/bertscore_cands.pkl bertscore_cands.pkl
!cp  ./drive/MyDrive/bertscore_exploded.pkl bertscore_exploded.pkl
!cp ./drive/MyDrive/chexpert_embeddings_train.npz chexpert_embeddings_train.npz
!cp ./drive/MyDrive/text_embeds_beam_search.npz text_embeds_beam_search.npz

Mounted at /content/drive


In [ ]:
!cp ./drive/MyDrive/chexpert_embeddings_train_final.npz chexpert_embeddings_train_final.npz


In [ ]:
import numpy as np
text_embeds = np.load('text_embeds_beam_search.npz')['a']

In [ ]:
im_embeds = np.load('chexpert_embeddings_train_final.npz')['a']

In [ ]:
import pandas as pd

df_exp = pd.read_pickle('bertscore_exploded.pkl')
df_cands = pd.read_pickle('bertscore_cands.pkl')
df_cands['num_cands'] = df_cands['cands'].apply(len)
df_cands['cum_num'] = df_cands['num_cands'].cumsum()
l = [0]
l.extend(list(df_cands['cum_num'])[:-1])
df_cands['start'] = l
df_cands['end'] = df_cands['cum_num']
df_cands['scores'] = df_cands.apply(lambda x:list(df_exp['score'][x['start']:x['end']]),axis=1)
dfc2 = df_cands.reset_index()[['index','cands','refs','scores']].explode(['cands','refs','scores']).reset_index(drop=True)
dfc2

,index,cands,refs,scores
0,0,No acute cardiopulmonary process.,No acute cardiopulmonary process.,1.0
1,0,No acute cardiopulmonary process. Bilateral no...,No acute cardiopulmonary process.,0.820996
2,0,1. No acute cardiopulmonary process. 2. Bilate...,No acute cardiopulmonary process.,0.742021
3,0,No evidence of pneumonia.,No acute cardiopulmonary process.,0.346208
4,0,1. No acute cardiopulmonary process. 2. Nodula...,No acute cardiopulmonary process.,0.723741
...,...,...,...,...
426590,125415,ET and NG tubes positioned appropriately. Biba...,Appropriately positioned ET and NG tubes. Biba...,0.931826
426591,125415,ET and NG tubes positioned appropriately.,Appropriately positioned ET and NG tubes. Biba...,0.565613
426592,125416,1. ET tube terminates approximately 2.9 cm fro...,Slight interval worsening of right lower lung ...,0.073246
426593,125416,Interval worsening of the right linear opacifi...,Slight interval worsening of right lower lung ...,0.682224


In [ ]:
dfc2 = dfc2.reset_index()
dfc2

,level_0,index,cands,refs,scores
0,0,0,No acute cardiopulmonary process.,No acute cardiopulmonary process.,1.0
1,1,0,No acute cardiopulmonary process. Bilateral no...,No acute cardiopulmonary process.,0.820996
2,2,0,1. No acute cardiopulmonary process. 2. Bilate...,No acute cardiopulmonary process.,0.742021
3,3,0,No evidence of pneumonia.,No acute cardiopulmonary process.,0.346208
4,4,0,1. No acute cardiopulmonary process. 2. Nodula...,No acute cardiopulmonary process.,0.723741
...,...,...,...,...,...
426590,426590,125415,ET and NG tubes positioned appropriately. Biba...,Appropriately positioned ET and NG tubes. Biba...,0.931826
426591,426591,125415,ET and NG tubes positioned appropriately.,Appropriately positioned ET and NG tubes. Biba...,0.565613
426592,426592,125416,1. ET tube terminates approximately 2.9 cm fro...,Slight interval worsening of right lower lung ...,0.073246
426593,426593,125416,Interval worsening of the right linear opacifi...,Slight interval worsening of right lower lung ...,0.682224


In [ ]:
mapper =dict(zip(dfc2['level_0'],dfc2['index']))
mapper

In [ ]:
im_embeds2 = np.vstack(list(map(lambda x:im_embeds[x,:],list(mapper.values()))))

In [ ]:
text_embeds.shape

(426595, 768)

In [ ]:
im_embeds2.shape

(426595, 1024)

In [ ]:
y = np.array(list(dfc2['scores']))

In [ ]:
y.shape

(426595,)

In [ ]:
import gc 
del df_exp
del df_cands
del im_embeds
del dfc2
gc.collect()

204

In [ ]:
X = np.hstack([text_embeds,im_embeds2])

In [ ]:
X.shape 

(426595, 1792)

In [ ]:
np.savez_compressed('scorer_train_data.npz',a=X)

In [ ]:
!cp scorer_train_data.npz ./drive/MyDrive/scorer_train_data.npz

In [ ]:
np.savez_compressed('scorer_train_data_targets.npz',a=y)

In [ ]:
!cp scorer_train_data_targets.npz ./drive/MyDrive/scorer_train_data_targets.npz